In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import preprocessing, neighbors, svm
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, VotingClassifier, GradientBoostingClassifier, ExtraTreesClassifier, \
  RandomForestRegressor, AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor, AdaBoostClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, r2_score,  confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression, LogisticRegressionCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import PCA

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from math import sqrt

import itertools
from itertools import cycle

import pickle
DIR_DATA = "data"
DIR_PROCESSED = "processed"
CV_FOLDS = 5

In [2]:
# train.json

In [3]:
df = pd.read_json(DIR_DATA + '/train.json')
df.head(20)

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],medium,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue
100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",high,40.7388,6887163,-74.0018,d9039c43983f6e564b1482b273bd7b01,[https://photos.renthop.com/2/6887163_de85c427...,2850,241 W 13 Street
100007,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,"[Hardwood Floors, No Fee]",low,40.7539,6888711,-73.9677,1067e078446a7897d2da493d2f741316,[https://photos.renthop.com/2/6888711_6e660cee...,3275,333 East 49th Street
100013,1.0,4,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,[Pre-War],low,40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,[https://photos.renthop.com/2/6934781_1fa4b41a...,3350,500 West 143rd Street
100014,2.0,4,38a913e46c94a7f46ddf19b756a9640c,2016-04-19 04:24:47,,West 18th Street,[],medium,40.7429,6894514,-74.0028,b209e2c4384a64cc307c26759ee0c651,[https://photos.renthop.com/2/6894514_9abb8592...,7995,350 West 18th Street
100016,1.0,2,3ba49a93260ca5df92fde024cb4ca61f,2016-04-27 03:19:56,Stunning unit with a great location and lots o...,West 107th Street,"[prewar, elevator, Dogs Allowed, Cats Allowed,...",low,40.8012,6930771,-73.9660,01287194f20de51872e81f660def4784,[https://photos.renthop.com/2/6930771_7e3622b6...,3600,210 West 107th Street
100020,2.0,1,0372927bcb6a0949613ef5bf893bbac7,2016-04-13 06:01:42,"This huge sunny ,plenty of lights 1 bed/2 bath...",West 21st Street,"[Doorman, Elevator, Pre-War, Terrace, Laundry ...",low,40.7427,6867392,-73.9957,e6472c7237327dd3903b3d6f6a94515a,[https://photos.renthop.com/2/6867392_b18283f6...,5645,155 West 21st Street
100026,1.0,1,a7efbeb58190aa267b4a9121cd0c88c0,2016-04-20 02:36:35,<p><a website_redacted,Hamilton Terrace,"[Cats Allowed, Dogs Allowed, Elevator, Laundry...",medium,40.8234,6898799,-73.9457,c1a6598437b7db560cde66e5a297a53f,[https://photos.renthop.com/2/6898799_3759be4c...,1725,63 Hamilton Terrace
100027,2.0,4,0,2016-04-02 02:58:15,This is a spacious four bedroom with every bed...,522 E 11th,"[Dishwasher, Hardwood Floors]",low,40.7278,6814332,-73.9808,23a01ea7717b38875f5b070282d1b9d2,[https://photos.renthop.com/2/6814332_e19a8552...,5800,522 E 11th


In [4]:
df = df.drop(['photos', 'description','latitude', 'longitude', 'display_address', 'street_address', 'listing_id', 'created'], 1)

In [5]:
df.building_id = df.building_id.astype('category').cat.codes
df.interest_level = df.interest_level.astype('category').cat.codes
df.manager_id = df.manager_id.astype('category').cat.codes


In [6]:
df.describe()

,bathrooms,bedrooms,building_id,interest_level,manager_id,price
count,49352.00000,49352.000000,49352.000000,49352.000000,49352.000000,4.935200e+04
mean,1.21218,1.541640,3223.338548,1.149741,1783.531995,3.830174e+03
std,0.50142,1.115018,2442.355834,0.531884,1006.158397,2.206687e+04
min,0.00000,0.000000,0.000000,0.000000,0.000000,4.300000e+01
25%,1.00000,1.000000,778.000000,1.000000,886.750000,2.500000e+03
50%,1.00000,1.000000,3210.500000,1.000000,1788.000000,3.150000e+03
75%,1.00000,2.000000,5340.000000,1.000000,2731.000000,4.100000e+03
max,10.00000,8.000000,7584.000000,2.000000,3480.000000,4.490000e+06


In [7]:
df.head()

,bathrooms,bedrooms,building_id,features,interest_level,manager_id,price
10,1.5,3,2431,[],2,1239,3000
10000,1.0,2,5862,"[Doorman, Elevator, Fitness Center, Cats Allow...",1,1583,5465
100004,1.0,1,5806,"[Laundry In Building, Dishwasher, Hardwood Flo...",0,2965,2850
100007,1.0,1,1201,"[Hardwood Floors, No Fee]",1,225,3275
100013,1.0,4,0,[Pre-War],1,2081,3350


In [8]:
print('Percent low: {:.2f}%'.format((np.sum(df.interest_level == 0) / len(df.interest_level)) * 100))
print('Percent medium: {:.2f}%'.format((np.sum(df.interest_level == 1) / len(df.interest_level)) * 100))
print('Percent high: {:.2f}%'.format((np.sum(df.interest_level == 2) / len(df.interest_level)) * 100))

Percent low: 7.78%
Percent medium: 69.47%
Percent high: 22.75%


In [ ]:
def getTime(appointment):
    hour, minute, _ = appointment.strip('Z').split(':')
    
    hour = int(hour[-2:])
    minute = int(minute)
    return 60*hour + minute

ma["AppointmenTime"] = ma.AppointmentRegistration.apply(lambda x: getTime(x))

In [ ]:
ma.describe()

In [ ]:
ma.Status = ma.Status.astype('category').cat.codes
ma.Gender = ma.Gender.astype('category').cat.codes
ma.DayOfTheWeek = ma.DayOfTheWeek.astype('category').cat.codes
# hr.Work_accident = hr.Work_accident.astype('category')
# hr.promotion_last_5years = hr.promotion_last_5years.astype('category')
# hr.left = hr.left.astype('category')

# Normalizing waiting time to mean 0 and unit variance
ma.AwaitingTime = ma.AwaitingTime.astype(np.float64)
wait_time_scaler = preprocessing.MinMaxScaler() #preprocessing.StandardScaler()
ma.AwaitingTime = wait_time_scaler.fit_transform(ma.AwaitingTime.values.reshape(-1,1))

# Feature Scaling Age to be between [0,1]
ma.Age = ma.Age.astype(np.float64)
age_scaler = preprocessing.MinMaxScaler()
ma.Age = age_scaler.fit_transform(ma.Age.values.reshape(-1,1))

# Feature Scaling Age to be between [0,1]
ma.AppointmenTime = ma.AppointmenTime.astype(np.float64)
time_scaler = preprocessing.MinMaxScaler()
ma.AppointmenTime = time_scaler.fit_transform(ma.AppointmenTime.values.reshape(-1,1))




In [ ]:
ma.head()

In [ ]:
ma.describe()

<!-- TEASER_END -->

In [ ]:
drop = ['Status','AppointmentRegistration','ApointmentData']
def predict_left(df, clf, test_size=0.2):
    X = df.drop(drop,1)
    y = df.Status 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    
    
    clf.fit(X_train, y_train)
    
    train_pred = clf.predict(X_train)
    test_pred = clf.predict(X_test)
    print('Training Score: {:.3f}'.format(clf.score(X_train, y_train)))
    print('Testing Score: %.3f' % (clf.score(X_test, y_test)))
    
    print()

In [ ]:
# Random Forest is bad at this, and K-NN is good but slow
classifiers = [neighbors.KNeighborsClassifier(n_jobs=-1), RandomForestClassifier(class_weight='balanced',n_jobs=-1), RandomForestClassifier(class_weight='balanced', criterion='entropy', n_jobs=-1),LogisticRegressionCV(class_weight='balanced', n_jobs=-1), AdaBoostClassifier(),xgb.XGBClassifier(nthread=-1), MultinomialNB()]#, neighbors.KNeighborsClassifier(n_jobs=-1)] #svm.SVC(), GradientBoostingClassifier(),  neighbors.KNeighborsClassifier(n_jobs=-1)], RandomForestClassifier(n_jobs=-1), RandomForestClassifier(criterion='entropy', n_jobs=-1),
# classifiers = [RandomForestClassifier(n_estimators=500 ,n_jobs=-1), RandomForestClassifier(n_estimators=500, criterion='entropy', n_jobs=-1), xgb.XGBClassifier(n_estimators=500, nthread=-1)]# svm.SVC()]

for i, clf in enumerate(classifiers):
    print('Classifier ', i)
    
    predict_left(ma, clf)

In [ ]:
def cross_val_left(df, clf, cv_folds=CV_FOLDS, drop=drop):
    X = df.drop(drop, 1)#, 'sales', 'salary'],1)
    y = df.Status 
    scores = cross_val_score(clf, X, y, cv=cv_folds, n_jobs=-1)
    
    
    
    print('Cross val score: ', sum(scores) / cv_folds )
    print(scores)
    
    print()
    

In [ ]:
classifiers = [xgb.XGBClassifier(n_estimators=500, nthread=-1), LogisticRegressionCV(class_weight='balanced', n_jobs=-1), AdaBoostClassifier(n_estimators=500), MultinomialNB()]#} RandomForestClassifier(n_estimators=500 ,n_jobs=-1), RandomForestClassifier(n_estimators=500, criterion='entropy', n_jobs=-1)]#, svm.SVC()]#]# svm.SVC()]

for i, clf in enumerate(classifiers):
    print('Classifier ', i)
    
    cross_val_left(ma, clf)

In [ ]:
clf = xgb.XGBClassifier(n_estimators=500, nthread=-1)
X = ma.drop(drop,1)
y = ma.Status 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    
clf.fit(X_train, y_train)
print(X.columns)
clf.feature_importances_
# Drop all lower than 0.01 relevance

In [ ]:
# Dropping all with <0.01 relevance seems to not affect score much (.9920->.9916)
drop = ['left', 'promotion_last_5years', 'Work_accident', 'sales', 'salary']
for i, clf in enumerate(classifiers):
    print('Classifier ', i)
    
    cross_val_left(hr, clf, drop=drop)

In [ ]:
ma.corr()["Status"]

In [ ]:
# Drop everything with corr to left of < 0.005
# Makes it worse
drop = ['left',  'sales', 'salary']
classifiers = [RandomForestClassifier(n_estimators=500 ,n_jobs=-1), RandomForestClassifier(n_estimators=500, criterion='entropy', n_jobs=-1)]#, svm.SVC()]#xgb.XGBClassifier(n_estimators=500, nthread=-1)]# svm.SVC()]

for i, clf in enumerate(classifiers):
    print('Classifier ', i)
    
    cross_val_left(hr, clf, drop=drop)

In [ ]:
ma.columns

In [ ]:
from sklearn.feature_selection import RFE
model = xgb.XGBClassifier(n_estimators=500, nthread=-1)#RandomForestClassifier(n_estimators=500, n_jobs=-1)
clf = RFE(model,5 )

X = ma.drop(drop,1)
y = ma.Status 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    
clf.fit(X_train, y_train)
print(clf.support_)
print(clf.ranking_)
# Gets same result as feature_importance, which makes sense

In [ ]:
# from sklearn.feature_selection import RFE
model = LogisticRegression()
clf = RFE(model,5 )

X = ma.drop(drop,1)
y = ma.Status 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    
clf.fit(X_train, y_train)
print(clf.support_)
print(clf.ranking_)
# Gets same result as feature_importance

In [ ]:
# PCA on relevant features
# drop = ['left',  'sales', 'salary']

y = np.array(ma.Status)

X = np.array(ma.drop(drop,1))
pca = PCA(n_components=2).fit(X)
X_pca = pca.transform(X)

In [ ]:
print(pca.components_)

In [ ]:
print(pca.explained_variance_ratio_)
print(sum(pca.explained_variance_ratio_))

In [ ]:
# Still get good same accuracy, ~69.7%
clf = LogisticRegressionCV(class_weight='balanced', n_jobs=-1)
y = np.array(ma.Status) 

scores = cross_val_score(clf, X_pca, y, cv=CV_FOLDS, n_jobs=-1)
    
    
    
print('Cross val score: ', sum(scores) / CV_FOLDS )
print(scores)

In [ ]:
colors = cycle('rb')
target_ids = range(2)
plt.figure()
for i, c, label in zip(target_ids, colors, ["Missed","Kept"]):
    plt.scatter(X_pca[y == i, 0], X_pca[y == i, 1],
                c=c, label=label)
plt.legend()
plt.show()

In [ ]:
# 3D PCA, Clear plane difference between them
# drop = ['left',  'sales', 'salary']

y = np.array(ma.Status)

X = np.array(ma.drop(drop,1))
pca = PCA(n_components=3).fit(X)
X_pca = pca.transform(X)

colors = cycle('rb')
target_ids = range(2)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for i, c, label in zip(target_ids, colors, ["Missed", "Kept"]):
    plt.scatter(X_pca[y == i, 0], X_pca[y == i, 1],X_pca[y == i, 2],
                c=c, label=label)
    
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
    
plt.legend()
plt.show()

In [ ]:
# 3D accuracy is the same as 2D
clf = RandomForestClassifier(class_weight='balanced',n_jobs=-1)#LogisticRegressionCV(class_weight='balanced', n_jobs=-1)
y = ma.Status 

scores = cross_val_score(clf, X_pca, y, cv=CV_FOLDS, n_jobs=-1)
    
    
    
print('Cross val score: ', sum(scores) / CV_FOLDS )
print(scores)

In [ ]:
clf = svm.SVC(class_weight='balanced')#LogisticRegression(class_weight='balanced', n_jobs=-1)
X = ma.drop(drop,1)
y = ma.Status 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_pred, y_test))

In [ ]:

class_names = ["No-Show","Show-Up"]
print(classification_report(y_test,y_pred, target_names=class_names))

In [ ]:
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    
    
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=10)


plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix')


plt.show()

In [ ]:
print(classification_report(y_test,y_pred, target_names=class_names))

In [ ]:
ma.columns

In [ ]:
percentageLeft = []
week = list(range(7))

for i in week:
    day = ma[ma.DayOfTheWeek == i]
    percentageLeft.append(np.sum(day.Status == 0) / len(day.Status) * 100)
    
plt.bar(week, percentageLeft)
plt.show()

In [ ]:
for c in ['Gender', 'Diabetes', 'Alcoolism', 'HiperTension',
       'Handcap', 'Smokes', 'Scholarship', 'Tuberculosis', 'Sms_Reminder', "AwaitingTime", "Age"]:
    percentageLeft = []
    genders = ma[c].unique()

    for i in genders:
        sex = ma[ma[c] == i]
        percentageLeft.append(np.sum(sex.Status == 0) / len(sex.Status) * 100)
    
    plt.bar(genders, percentageLeft)
    plt.show()